<img src="https://tinyurl.com/k2t79s6t" style="float: left; margin: 20px; height: 55px">

# Python 105: Data Analytics Salary Analysis
 _Author: Martin Arroyo_



## Scenario
You are an analyst working on a new data set that your company acquired for research purposes. It contains salary information for data analytics professionals collected between 2020 and 2022.

Management has asked you to do an exploratory analysis of this new data. [Click here to read the email they sent you.](https://docs.google.com/document/d/1J-3SG76KQxj-aeKQ6-ZmRuntsAdoPqC8lMtQPzuhVJQ/edit?usp=sharing)


## Task

### Part One
Clean the `ds_salaries` table using the outline provided below. Use documentation to refresh your memory on syntax. After you're done cleaning, your data should have no null values and no duplicates.
### Part Two
Complete the analysis using everything we've learned so far in Python 101-104. Ensure that the data is clean, then answer the questions in the code cells under **Part Two**. Use a combination of visuals, tabular data, and written language to answer the questions. **Create at least 3 visuals**.
### Part Three
After you're done, write a response to your manager's email letting them know that you have completed your assignment. Share any additional insights or findings you may have found in the data, along with:

- a copy of your notebook with your code and visuals
- the `ds_salaries_clean` table as a `csv` file

This should be a professional email response. In this scenario, your pod captain will play the role of manager.

<hr />


## Tips

### Using Python + SQL

Since we are connected to a database, you can use a combination of `SQL` and `Python` to answer your questions.

To query the database directly, you can use the `%%sql` cell magic at the top of a cell, then write your query below like this:
```sql
%%sql

SELECT * FROM ds_salaries;
```

Once you have the results you like, copy and paste the query into a `string`, then get your result as a `DataFrame` using `read_sql`:
```python
query = "SELECT * FROM ds_salaries"
my_query_df = pd.read_sql(query, con=engine)
```

### Keep in Mind

This is not a client project, nor will this be part of a presentation. It should be neat, but any visuals you make do not need to be fancy.

### Best Practices

If you get stuck, first do a search for the problem you're trying to solve. More often than not, someone else has encountered a similar issue and you'll be able to find a relatively clear answer. You can also check the documentation of the program you're using. If you find yourself getting frustrated, stop and take a short break. If all else fails, then reach out to another person for help.

### Troubleshooting

There may be things here that you don't remember how to do, or maybe there is something you want to do that we haven't covered. Use the resources provided to you in Slack and from the other Python nights, the Python 101-104 notebooks, as well as whatever you can find on the internet to help you.

Here are a few to get you started:
- [w3schools](https://www.w3schools.com/python/)
- [DataCamp Cheat Sheets](https://www.datacamp.com/cheat-sheet)
- [Stack Overflow](https://stackoverflow.com/)
- [SQL Basics Cheatsheet](https://martinmarroyo.github.io/sqlcheatsheetandresources-coop/)

## Connect to the Database and get the data

Run the following cell to establish the connection to your team's database and get the data (this may take a minute or two to complete):

In [ ]:
# Install `teachdb` if it's not in the system already
from importlib.util import find_spec
if not find_spec('teachdb'):
    print("Installing `teachdb` and its dependencies...")
    !pip install --quiet --upgrade git+https://github.com/freestackinitiative/teachingdb.git
    print("Successfully installed `teachdb`")
import os
import duckdb
import pandas as pd
import sqlalchemy as sql
import matplotlib.pyplot as plt
from teachdb.teachdb import connect_teachdb

# Set configurations for notebook
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Load data
engine = duckdb.connect(":memory:")
connect_teachdb(engine, database="ds_salaries")

%sql engine

# Get the data from the database into a DataFrame
ds_salaries = pd.read_sql("SELECT * FROM ds_salaries", con=engine)
countries = pd.read_sql("SELECT * FROM countries", con=engine)
usd_exchange_rates = pd.read_sql("SELECT * FROM usd_exchange_rates", con=engine)
experience_levels = pd.read_sql("SELECT * FROM experience_levels", con=engine)
employment_types = pd.read_sql("SELECT * FROM employment_types", con=engine)

## Part One: Cleaning the data

Our task is to explore this new dataset, clean it, then answer the questions from management using the data. We haven't seen the data before, so our first step is familiarize ourselves with it.

Since the data has not been cleaned, we consider it to be in its "raw" form. It's good practice to not modify raw data directly - that way you can revert back to the original state when/if you need to. Let's start by making a copy of `ds_salaries`:

In [2]:
ds_salaries_clean = ds_salaries.copy()

In [3]:
ds_salaries_clean

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,remote_work_type,company_location,company_size,job_category
0,2020,MI,FT,Data Scientist,70000.0,EUR,DE,0,On-site only,DE,L,Data Science
1,2020,SE,FT,Machine Learning Scientist,260000.0,USD,JP,0,On-site only,JP,S,Machine Learning
2,2020,SE,FT,Big Data Engineer,85000.0,GBP,GB,50,Hybrid,UK,M,Data Engineering
3,2020,MI,FT,Product Data Analyst,20000.0,USD,HN,0,On-site only,HN,S,Data Analyst
4,2020,SE,FT,Machine Learning Engineer,150000.0,USD,US,50,Hybrid,US,L,Machine Learning
...,...,...,...,...,...,...,...,...,...,...,...,...
1251,2022,MI,FT,Data Engineer,45000.0,GBP,GB,100,Fully remote,UK,M,Data Engineering
1252,2022,SE,FT,Data Scientist,260000.0,USD,US,100,Fully remote,US,M,Data Science
1253,2022,SE,FT,Data Science Engineer,60000.0,USD,AR,100,Fully remote,MX,L,Data Engineering
1254,2022,MI,FT,Data Engineer,63900.0,USD,US,0,On-site only,US,M,Data Engineering


Now that we have our copy, let's begin exploring the data and its structure. Use the summary functions that we learned in `Python 103` to look at the metadata for `ds_salaries_clean` and observe the number of rows, columns, datatypes, and null values:


Check the first 5 rows of the data using the `head` method:

Find the total number of null values per column using the `isna` method:

After looking more closely, we see that there are null values in some columns. Management doesn't want any null values in the cleaned data. But if we just drop all the rows with null values, we'll lose data unneccessarily. We want to keep any data we can and discard only the records we can't reasonably salvage.

Let's start with the `salary` column, which has 14 null values. Can we reasonably infer what those missing salaries are?

While it's possible to guess what those missing values could be, the results are not guaranteed to be accurate. This could skew our analysis. Also, the rows with missing salaries have missing job titles too, which we would want to know as well. Since there are only 14 rows (~2% of the total data) that are affected, the best approach for our purposes would be to just drop them.

Drop the rows with null values in the `salary` column:

Now verify that the rows in the salary column were dropped by checking the null values in `ds_salaries_clean` again:

#### Test
Run the following cell to ensure that you dropped the rows correctly before continuing (There should be no output if correct):

In [ ]:
assert ds_salaries_clean.salary.isna().sum() == 0

Now that we've taken care of removing those rows, let's look at the other columns with null values. The `company_size` column has 8 null values. This is another case where we could possibly guess at these values, but for our purposes its better to drop rows where this is null.

Go ahead and drop all rows where `company_size` is null:

#### Test

Run the following code cell to check your work:

In [ ]:
assert ds_salaries_clean.company_size.isna().sum() == 0

We've handled most of the null values in our data so far. The last column that we have to deal with is `remote_work_type`, which has 41 nulls. If we look at both `remote_work_type` and `remote_ratio`, we should see that `remote_work_type` is directly related to the `remote_ratio`. We can determine what the work type is by using the ratio. Since we have all the ratios and are only missing some types, we can fill in the nulls using the `remote_ratio`.

Use the example below, which fills in the values for `On-site only` types, to fill in the nulls for `Hybrid` and `Fully remote`:

Run the next cell:

In [ ]:
# Create a filter for "On-site only" remote ratios
onsite_only = ds_salaries_clean.loc[ds_salaries_clean.remote_ratio == 0]
# Set the `remote_work_type` values for the data found by the filter to "On-site only"
ds_salaries_clean.loc[onsite_only.index, 'remote_work_type'] = 'On-site only'

In [ ]:
# Your Answer here

#### Test

After filling in the missing values for `remote_work_type`, there should be no more null values in the data. Run the cell below to confirm that your work is correct so far:

In [ ]:
# There are no nulls in remote_work_type
assert ds_salaries_clean.remote_work_type.isna().sum() == 0
# The values in remote_work_type were created correctly
values = list(ds_salaries_clean.remote_work_type.unique())
values_check = ['On-site only', 'Fully remote', 'Hybrid']
assert  len(values) == len(values_check) and len([i for i in values if i in values_check]) == 3
# There are no more null values in ds_salaries_clean
assert ds_salaries_clean.isna().sum().sum() == 0

Great work!

The nulls are taken care of, but there is an issue with the `salary` column - these salaries are all in different currencies! Good thing we have the `usd_exchange_rates` table. We can use that data to convert all the `salary` information in `ds_salaries_clean` to USD for consistency.

First, familiarize yourself with `usd_exchange_rates` by looking at the data and checking the column definitions in the Data Dictionary supplied by management:

Now that you're familiar with the `usd_exchange_rates`, it's time to join it to `ds_salaries_clean`. Fill in the code below to join the two tables:

In [ ]:
ds_salaries_clean = pd.function_to_join(
    ds_salaries_clean,
    usd_exchange_rates,
    left_on = ["Columns to join from ds_salaries_clean"],
    right_on = ["Columns to join from usd_exchange_rates"]
)

With our data merged, it's time to convert the salary. Create a new column called `salary_in_usd` that is the result of the `salary` divided by `exchange_rate`.

Example:

```python
# Note: The `round` function can be used to round your results
df.loc[:, 'new_column'] = round(df['col1'] / df['col2'])
```

Great! Now we have a uniform way to measure the salary information.

Before we finish cleaning, there are a few columns with abbreviations (`experience_level`, `employment_type`, and `company_location`). We should have the full names of these terms as columns in the cleaned data set.

Using the data dictionary and the `merge` function, join the `experience_levels`, `employment_types`, and `countries` data into `ds_salaries_clean` to translate the abbreviations:

Example:
```python
df = pd.merge(df, df2, left_on="df_col", right_on="df2_col")
```

Ok, now we're almost done cleaning. But if we look at `ds_salaries_clean`, there are some extra columns we don't need that were added from the joins we just did.

Drop the following columns:

`abbreviation_x`,`abbreviation_y`, `abbreviation`, `iso_code`, `ref_date`

#### Test

Run the cell below to confirm the columns were dropped correctly:

In [ ]:
dropped_cols = ['abbreviation_x','abbreviation_y', 'abbreviation', 'iso_code', 'ref_date']
assert len([i for i in ds_salaries_clean if i in dropped_cols]) == 0

We've handled all the nulls, added columns with more information, and mostly cleaned our data. The only thing that's left is to remove any duplicate rows. Go ahead and drop the duplicates using the `drop_duplicates` method on `ds_salaries_clean`:

Great work!

Now the data should be clean and ready for analysis! Run the next cell as one last test before writing the new table as a `csv` and starting your analysis:


#### Test

In [ ]:
# Test for null values
assert ds_salaries_clean.isna().sum().sum() == 0
# Test for duplicates
assert ds_salaries_clean.duplicated().sum() == 0

If the tests in the cell above pass, you're ready to do your analysis! Let's write the cleaned table to a csv file first:

In [ ]:
"""
This line is meant to write their `clean` dataframe to a csv file for submission. This will save
the file in the same folder where this notebook is located. The file name will be `ds_salaries_clean.csv`.

For students using Google Colab, the file will be saved to the folder space accessed via the menu on the
left side of the screen. They can download the file to their local machine from there.
"""

ds_salaries_clean.to_csv("ds_salaries_clean.csv", index=False)

## Part Two: Analysis

Now it's time to start analyzing the data and answering the questions from management. If you haven't done so yet, familiarize yourself with the questions and data dictionary to get a better sense of the data.

The questions asked mostly involve summarizing parts of the data and reporting your findings. You will be doing a lot of aggregate functions (get your `groupby` ready!)

Unlike the previous section, this one is not as guided. You will get some hints, but you are expected to answer the questions using your existing `Python` and/or `SQL` knowledge along with help from Google and the other resources provided.

You can use `SQL` to query the data to get the answers, or you can use `Python` only, or a combination of both.

There are only two rules:

1. You must create at least 3 visuals using `matplotlib`
2. If you answer a question using `SQL` only, still create a new `pandas` DataFrame that has the results of your query

 Example:
 ```python
 # Create a new DataFrame from a SQL query
 my_query = """
      SELECT *
      FROM ds_salaries
 """
 new_df = pd.read_sql(my_query, con=engine)
 ```

Now on to the questions!


*Note: There is a table in the database called `ds_salaries_clean` that is a copy of the cleaned table that you should use for your queries if you're using `SQL`*

### What is the average overall salary (regardless of year) by job category?

>*Hint*:
>
>*One of the keywords in this question is `by`. When you see this, it indicates that you'll need to summarize the data by the field(s) that comes after `by`. You'll have to use `groupby` and an aggregate function to solve this.*
>
>*Make sure to order the results so that it's easy to see the largest salaries vs the smallest. This way answers are clearly visible.*
>
>*Also, use the `round` function to format your salary results to make them look neater!*


### What is the average salary by job category and experience level (regardless of year)?

>*Hint*:
>
>*This is similar to the first question, except instead of grouping by one column, you're grouping by two. Order your results by `job_category`*
>
>*Also, use the `round` function to format your salary results to make them look neater!*
>
>*This will be tough to make a visual for since we didn't go over subplots. If you want to challenge yourself, go ahead and look up tutorials on how to do this! But be mindful of time and consider using a table to visualize this*

### In which country do data analysts make the most money on average? Which country do they make the least?

>*Hint*:
>
>*This is a filtering and aggregation problem. While we don't have the `by` keyword here, the question asks us to look at average salary for data analysts in each country. We are  filtering for data analysts and grouping salaries by country.*
>
>*Filter the data by `Data Analyst` and then get the average salary by country for that subset. Finally, we want to see the min and max values for this subset. Solving to get just the min and max values is a bit more complex than what we've learned so far. It is OK to show the entire list (ordered, of course) and highlight the highest and lowest values in your response.*

### Which job category earns more on average: Data Analyst, Data Science, or Data Engineering?

>*Hint*:
>
>*Another filtering and aggregation problem. This time we're looking at average salary by specific job categories. Use the same pattern as the previous filtering/aggregation questions that we answered to complete this.*

### Which company size pays data professionals the most on average? Is there a relationship between company size and average pay?

>*Hint:*
>
>*This is an aggregation problem with an additional question that can only be answered following the aggregation. Follow the same pattern as the previous questions and use the results to determine whether there is a relationship between pay and company size.*

### Which job category pays the most on average in 2022?

>*Hint:*
>
>*This is another filtering and aggregation problem. Apply the same pattern you've used previously to solve this. You should return only the job category with the highest average pay in 2022*

### How are companies working in 2022 (Remote, In-Office, Hybrid)?

>*Hint:*
>
>*This is also a filtering and aggregation problem (a fairly common theme.) Apply the same pattern you've used previously to solve this.*

### What additional analysis/insights can you make using the dataset?

>*Hint:*
>
>*Here's your chance to provide management with unique input or insight based on what you have found in the database to management.*
>
>*This is an optional question, but we encourage you to explore the data for relationships or patterns that were not asked about but may be relevant.*

In [ ]:
"""
This is an optional and totally open ended question. It's not necessarily meant to be graded but rather
to challenge students to think beyond the outline we gave them
"""

## Part Three: Delivering Results

You've cleaned the data, answered your manager's questions, and created some visualizations to share your insights. Job well done! Now its time to send the results to your manager.

Compose an email response to your manager's request. Provide an overview of the analysis you did and the materials you are submitting for their review. Attach a copy of the `ds_salaries_clean` data set, as well as this notebook, to the email and send it to your pod captain.